In [1]:
# Julia 1.8.4
using Pkg
Pkg.activate(".")
Pkg.instantiate()
# Pkg.add("CUDA")
# Pkg.add("Glob")
# Pkg.add("Flux")
# Pkg.add("NIfTI")
# Pkg.add("Images")
# Pkg.add("FastAI")
# Pkg.add("Metalhead")
# Pkg.add("FastVision")
# Pkg.add("CairoMakie")
# Pkg.add("StaticArrays")
# Pkg.add("MLDataPattern")
# Pkg.add("BenchmarkTools")
# Pkg.add("ChainRulesCore")
# Pkg.add("BSON")
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add(url = "https://github.com/Dale-Black/ComputerVisionMetrics.jl")
# Pkg.add(url="https://github.com/Dale-Black/DistanceTransforms.jl", rev="wenbo")

  Activating project at `c:\Users\wenbl13\OneDrive - UCI Health\Desktop\paper\hd-loss\notebooks`

In [2]:
using CUDA
using Glob
using Dates
using NIfTI
using Images
using Statistics
using StaticArrays
using MLDataPattern
using BenchmarkTools
using ChainRulesCore
using DistanceTransforms
using Printf
using DataFrames
using CSV
using ComputerVisionMetrics
using FastAI, FastVision, Flux, Metalhead
import CairoMakie; CairoMakie.activate!(type="png")

# Load and prepare data

In [3]:
mask_dir = raw"E:\RSNA_compressed_data\96_96_96\segmentations"
train_image_dir = raw"E:\RSNA_compressed_data\96_96_96\train_images"
mask_names = []
for f in readdir(mask_dir, join=false)
    push!(mask_names, splitext(splitext(f)[1])[1])
end

matched_names = []
ct = 0
for img in readdir(train_image_dir, join=false)
    f = splitext(img)[1]
    if f in mask_names
        push!(matched_names, f)
        ct += 1
    end
end
println("Found $ct of $(size(mask_names)[1])")

image_size = (96, 96, 96)
container_images = Array{Float32, 4}(undef, image_size..., ct)
container_masks = Array{Float32, 4}(undef, image_size..., ct)
# Load 87 images from saved data.
Threads.@threads for i = 1 : ct
    curr_path = @sprintf "%s\\%s.txt" train_image_dir matched_names[i]
    temp = zeros(image_size)
    read!(curr_path, temp)
    container_images[:,:,:, i] = deepcopy(temp)
end
# Normalize images
for i = 1 : 87
    curr_img = container_images[:,:,:, i]
    curr_max, curr_min = maximum(curr_img), minimum(curr_img)
    curr_img = (curr_img .- curr_min) ./ (curr_max - curr_min)
    container_images[:,:,:, i] = curr_img
end
# Load 87 masks from saved data.
Threads.@threads for i = 1 : ct
    curr_path = @sprintf "%s\\%s.nii.txt" mask_dir matched_names[i]
    temp = zeros(image_size)
    read!(curr_path, temp)
    container_masks[:,:,:, i] = deepcopy(temp)
end
container_masks = Int64.(round.(container_masks) .+ 1)
data_ready = (container_images, container_masks);

image1, mask1 = sample = (container_images[:,:,:,1], container_masks[:,:,:,1]);

data_mean, data_std = FastVision.imagedatasetstats(container_images, Gray{N0f8}) 
data_mean, data_std = StaticArraysCore.SVector{1, Float32}(data_mean[1]), StaticArraysCore.SVector{1, Float32}(data_std[1])

task = SupervisedTask(
    (FastVision.Image{3}(), Mask{3}(1:8)), # 1: background, 2-8: 7 classes
    (
        FastVision.ProjectiveTransforms((image_size)),
        ImagePreprocessing(means = data_mean, stds = data_std, C = Gray{N0f8}),
        FastAI.OneHot()
    )
)
println("Preparing Dataset...")
batch_size = 1

@assert checkblock(task.blocks.sample, sample)
train_files, val_files = MLDataPattern.splitobs(data_ready, 0.8)
tdl, vdl = FastAI.taskdataloaders(train_files, val_files, task, batch_size);

Found 87 of 87


Progress:   2%|█                                        |  ETA: 0:01:38

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


Preparing Dataset...


# Get metrics

## Dice Model

In [4]:
dice_metric_of_dice_model_train_set = []
hd_metric_of_dice_model_train_set = []

dice_metric_of_dice_model_valid_set = []
hd_metric_of_dice_model_valid_set = []

for epoch_idx = 10 : 10 : 500
    print("$epoch_idx...\t")
    # Load models
    DICE_model_path = string("E:/RSNA_savedmodels/bigger_NN_0.001_Dice_", epoch_idx, ".jld2")
    _ , model_Dice = loadtaskmodel(DICE_model_path)
    _ , model_Dice = nothing, model_Dice |> gpu
    # Get metrics

    # Traverse train dataloader
    curr_dice_metric_of_dice_model = []
    curr_hd_metric_of_dice_model = []
    for (_img, _mask) in tdl
        mask = _mask .>= 0.5
        img = _img |> gpu

        # Predictions
        pred_mask_DICE = (model_Dice(img) |> cpu) .>= 0.5

        curr_curr_dice_metric_of_dice_model = Array{Any}(undef, 7)
        curr_curr_hd_metric_of_dice_model = Array{Any}(undef, 7)

        Threads.@threads for channel_idx = 2:8
            # Dice metric for Dice model
            curr_curr_dice_metric_of_dice_model[channel_idx-1] = ComputerVisionMetrics.dice(pred_mask_DICE[:,:,:,channel_idx,1], mask[:,:,:,channel_idx,1])
    
            # HD metric for Dice model
            curr_curr_hd_metric_of_dice_model[channel_idx-1] = ComputerVisionMetrics.hausdorff(pred_mask_DICE[:,:,:,channel_idx,1], mask[:,:,:,channel_idx,1])
        end
        # Record
        append!(curr_dice_metric_of_dice_model, curr_curr_dice_metric_of_dice_model)
        append!(curr_hd_metric_of_dice_model, curr_curr_hd_metric_of_dice_model)
    end
    push!(dice_metric_of_dice_model_train_set, mean(curr_dice_metric_of_dice_model))
    push!(hd_metric_of_dice_model_train_set, mean(curr_hd_metric_of_dice_model))


    curr_dice_metric_of_dice_model = []
    curr_hd_metric_of_dice_model = []
    # Traverse valid dataloader
    for (_img, _mask) in vdl
        mask = _mask .>= 0.5
        img = _img |> gpu

        # Predictions
        pred_mask_DICE = (model_Dice(img) |> cpu) .>= 0.5

        curr_curr_dice_metric_of_dice_model = Array{Any}(undef, 7)
        curr_curr_hd_metric_of_dice_model = Array{Any}(undef, 7)

        Threads.@threads for job_idx = 1:14
            if job_idx > 7
                # Dice metric for Dice model
                curr_curr_dice_metric_of_dice_model[job_idx - 7] = ComputerVisionMetrics.dice(
                    pred_mask_DICE[:,:,:,job_idx-6,1], mask[:,:,:,job_idx-6,1])
            else
                # HD metric for Dice model
                curr_curr_hd_metric_of_dice_model[job_idx] = ComputerVisionMetrics.hausdorff(
                    pred_mask_DICE[:,:,:,job_idx+1,1], mask[:,:,:,job_idx+1,1])
            end
        end
        # Record
        append!(curr_dice_metric_of_dice_model, curr_curr_dice_metric_of_dice_model)
        append!(curr_hd_metric_of_dice_model, curr_curr_hd_metric_of_dice_model)
    end
    push!(dice_metric_of_dice_model_valid_set, mean(curr_dice_metric_of_dice_model))
    push!(hd_metric_of_dice_model_valid_set, mean(curr_hd_metric_of_dice_model))

    # model_Dice = nothing

    curr_dice_metric_of_dice_model = nothing
    curr_hd_metric_of_dice_model = nothing
    
    # GC.gc(true)
    # CUDA.reclaim()
end
println()

10...	

## HD+Dice Model

In [ ]:
dice_metric_of_HD_Dice_model_train_set = []
hd_metric_of_HD_Dice_model_train_set = []

dice_metric_of_HD_Dice_model_valid_set = []
hd_metric_of_HD_Dice_model_valid_set = []

for epoch_idx = 10 : 10 : 500
    print("$epoch_idx...\t")
    # Load models
    HD_DICE_model_path = string("E:/RSNA_savedmodels/3_bigger_NN_0.001_HD_Dice_", epoch_idx, ".jld2")
    _ , model_HD_Dice = loadtaskmodel(HD_DICE_model_path)
    _ , model_HD_Dice = nothing, model_HD_Dice |> gpu
    # Get metrics

    # Traverse train dataloader
    curr_dice_metric_of_HD_Dice_model = []
    curr_hd_metric_of_HD_Dice_model = []
    for (_img, _mask) in tdl
        mask = _mask .>= 0.5
        img = _img |> gpu

        # Predictions
        pred_mask_HD_DICE = (model_HD_Dice(img) |> cpu) .>= 0.5

        curr_curr_dice_metric_of_HD_Dice_model = Array{Any}(undef, 7)
        curr_curr_hd_metric_of_HD_Dice_model = Array{Any}(undef, 7)
        
        Threads.@threads for job_idx = 1:14
            if job_idx > 7
                # Dice metric for Dice model
                curr_curr_dice_metric_of_dice_model[job_idx - 7] = ComputerVisionMetrics.dice(
                    pred_mask_DICE[:,:,:,job_idx-6,1], mask[:,:,:,job_idx-6,1])
            else
                # HD metric for Dice model
                curr_curr_hd_metric_of_dice_model[job_idx] = ComputerVisionMetrics.hausdorff(
                    pred_mask_DICE[:,:,:,job_idx+1,1], mask[:,:,:,job_idx+1,1])
            end
        end
        # Record
        append!(curr_dice_metric_of_HD_Dice_model, curr_curr_dice_metric_of_HD_Dice_model)
        append!(curr_hd_metric_of_HD_Dice_model, curr_curr_hd_metric_of_HD_Dice_model)
    end
    push!(dice_metric_of_HD_Dice_model_train_set, mean(curr_dice_metric_of_HD_Dice_model))
    push!(hd_metric_of_HD_Dice_model_train_set, mean(curr_hd_metric_of_HD_Dice_model))


    curr_dice_metric_of_dice_model = []
    curr_hd_metric_of_dice_model = []
    curr_dice_metric_of_HD_Dice_model = []
    curr_hd_metric_of_HD_Dice_model = []
    # Traverse valid dataloader
    for (_img, _mask) in vdl
        mask = _mask .>= 0.5
        img = _img |> gpu

        # Predictions
        pred_mask_HD_DICE = (model_HD_Dice(img) |> cpu) .>= 0.5

        curr_curr_dice_metric_of_HD_Dice_model = Array{Any}(undef, 7)
        curr_curr_hd_metric_of_HD_Dice_model = Array{Any}(undef, 7)
        
        Threads.@threads for channel_idx = 2:8
            # Dice metric for HD+Dice model
            curr_curr_dice_metric_of_HD_Dice_model[channel_idx-1] = ComputerVisionMetrics.dice(pred_mask_HD_DICE[:,:,:,channel_idx,1], mask[:,:,:,channel_idx,1])
    
            # HD metric for HD+Dice model
            curr_curr_hd_metric_of_HD_Dice_model[channel_idx-1] = ComputerVisionMetrics.hausdorff(pred_mask_HD_DICE[:,:,:,channel_idx,1], mask[:,:,:,channel_idx,1])
        end
        # Record
        append!(curr_dice_metric_of_HD_Dice_model, curr_curr_dice_metric_of_HD_Dice_model)
        append!(curr_hd_metric_of_HD_Dice_model, curr_curr_hd_metric_of_HD_Dice_model)
    end
    push!(dice_metric_of_HD_Dice_model_valid_set, mean(curr_dice_metric_of_HD_Dice_model))
    push!(hd_metric_of_HD_Dice_model_valid_set, mean(curr_hd_metric_of_HD_Dice_model))

    # model_HD_Dice = nothing

    curr_dice_metric_of_HD_Dice_model = nothing
    curr_hd_metric_of_HD_Dice_model = nothing
    
    # GC.gc(true)
    # CUDA.reclaim()

end
println()

# Record

In [ ]:
epoch_idx = collect(10 : 10 : 500)
df_metrics = DataFrame(epoch_idx = epoch_idx,
    dice_metric_of_dice_model_train_set = dice_metric_of_dice_model_train_set,
    hd_metric_of_dice_model_train_set = hd_metric_of_dice_model_train_set,
    dice_metric_of_HD_Dice_model_train_set = dice_metric_of_HD_Dice_model_train_set,
    hd_metric_of_HD_Dice_model_train_set = hd_metric_of_HD_Dice_model_train_set,

    dice_metric_of_dice_model_valid_set = dice_metric_of_dice_model_valid_set,
    hd_metric_of_dice_model_valid_set = hd_metric_of_dice_model_valid_set,
    dice_metric_of_HD_Dice_model_valid_set = dice_metric_of_HD_Dice_model_valid_set,
    hd_metric_of_HD_Dice_model_valid_set = hd_metric_of_HD_Dice_model_valid_set
    )
CSV.write("E:/CSVs/metrics_RSNA.csv", df_metrics)